In [21]:
import simpy
from __future__ import division

## Simpy tutorial

### Generator statements

In [22]:
# a generator creating the next natural number (as += 1) starting at i
def range_(n):
    i = 0
    while i < n:
        yield i
        i += 1

In [23]:
# for statement in the function
for value in range_(10):
    print(value)

0
1
2
3
4
5
6
7
8
9


In [24]:
# generator creates a generator object
x = range_(10)
print(x)

<generator object range_ at 0x106f765f0>


In [25]:
# next deliveres the next item in an iterator (not a list), does not 
print(next(x))

0


### Simpy basics

In [26]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [27]:
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


### In classes

In [28]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
            yield self.env.timeout(duration)

In [29]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [30]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env,car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


### Resources/Servers

In [31]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)

    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [32]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


### Generator classes

In [33]:
import numpy as np

class Server(object):
    """Server with constant service times""" 
    
    def __init__(self, env):
        self.env = env
        self.server = simpy.Resource(env, capacity=2)
    
    def server_time(self): 
        return np.random.exponential(5,1)
    
    def serve(self):
        serve_time = self.server_time()
        yield self.env.timeout(serve_time)
    
    _server_time = server_time
    _variabele = "Hoi"
    
class Server2(Server):
    
    def server_time(self):
        return 2

    

In [34]:
env = simpy.Environment()
sv1  = Server(env)
sv2 = Server2(sv1)

print(sv1.server_time())
print(sv2.server_time())
print(sv2._server_time())
print(sv2._variabele)



[10.16337225]
2
[2.75142865]
Hoi


In [70]:
class Customer(object):
    """
    For each customer we record the arrivaltime, 
    the response time and the departure time.

    """
    def __init__(self, env, sv):
        self.env = env
        self.sv = sv
        self.serviceTimes = []
        
    def enqueue(self):
        print("Arrived at %f" % (self.env.now))
        with self.sv.server.request() as request:
            yield request
            print("Started serving at %f" % (self.env.now))
            serviceStart = self.env.now
            print 'passed first definition'
            print serviceStart
            print serviceStart[0]
            print('serviceStart = %f' (serviceStart[0]))
            yield self.env.process(self.sv.serve())
            print("Finished serving at %f" % (self.env.now[0]))
            serviceEnd = self.env.now
            print('serviceEnd = %f' (serviceEnd))
            self.serviceTimes.append(serviceEnd[0] - serviceStart[0])
            

In [71]:
class CustomerGenerator(object):
    """Creates new customers"""
    def __init__(self,env,sv):
        self.env = env
        self.sv = sv
        self.max_customers = 2
        
    
    def arrival_time(self):
        return np.random.exponential(2,1)
    
    def run(self):
        customers = 0
        while customers < self.max_customers:
            arrival_time = self.arrival_time()
            yield self.env.timeout(arrival_time)
            
            customers += 1
            new_customer = Customer(self.env, self.sv)
            self.env.process(new_customer.enqueue())

In [72]:
class QTExperiment(object):
    """ Easy interface to generate a queue """
    
    
    def __init__(self):
        self.env = simpy.Environment()
        self.server = Server(self.env)
        self.generator = CustomerGenerator(self.env, self.server)
        
        
    def run(self):
        self.env.process(self.generator.run())
        self.env.run()

In [73]:
exp = QTExperiment().run()

Arrived at 0.799167
Started serving at 0.799167
passed first definition
[0.79916664]
0.7991666386211151


TypeError: 'str' object is not callable

In [49]:
customers = Customer(env, sv1)
print customers.serviceTimes
print customers.bullshit

[]


AttributeError: 'Customer' object has no attribute 'bullshit'